In [ ]:
import cv2
from matplotlib import pyplot
import numpy

# Ustawienie rozmarów wyświetlanych obrazów
pyplot.rcParams["figure.figsize"] = (18, 18)

a = numpy.array([1, 2, 3], dtype=numpy.int16)
print(a[0] - a[1])

In [ ]:
#
# Wczytanie obrazów źródłowych
#
# źródło grafiki colors.jpg: https://unsplash.com/photos/gT5kuls6Y6Q
# źródło grafiki gacław-na-kuchni.jpg: własne
#
#image_from_file = cv2.imread('images/colors.jpg')
image_from_file = cv2.imread('images/gacław-na-kuchni.jpg')
image_gray = cv2.cvtColor(image_from_file, cv2.COLOR_BGR2GRAY)
image_color = cv2.cvtColor(image_from_file, cv2.COLOR_BGR2RGB)
print('Rozmiar obrazka: ', image_from_file.shape)

# Zadanie 1

In [ ]:
def find_closest_palette_color(value, k):
    return round(value / 255) * 255


#
# Algorytm
#
def dither_image(input_img, k=2):
    output = numpy.array(input_img, dtype=numpy.int16)
    dither_matrix = numpy.array([[0, 0, 7], [3, 5, 1]]) / 16

    for y in range(output.shape[0] - 1):
        for x in range(1, output.shape[1] - 1):
            oldpixel = output[y, x]
            newpixel = find_closest_palette_color(oldpixel, k)
            output[y, x] = newpixel

            quant_error = oldpixel - newpixel

            residue = (dither_matrix * quant_error)
            residue += output[y:y+2,x-1:x+2]
            numpy.clip(residue, 0, 255, out=residue)
            residue = residue.astype(numpy.uint8)

            output[y:y+2,x-1:x+2] = residue
    return output.astype(numpy.uint8)



In [ ]:
dithered_gray = dither_image(image_gray)

#
# Wyświetlenie
#
_, plots = pyplot.subplots(1,2)

plots[0].imshow(image_gray, cmap='gray')
plots[1].imshow(dithered_gray, cmap='gray')

In [ ]:
#
# Histogram
#
pyplot.rcParams["figure.figsize"] = (10, 6)
histr = cv2.calcHist([dithered_gray], [0], None, [256], [0, 256])
pyplot.plot(histr)
pyplot.xlim([-1, 256])
pyplot.xlabel('Wartośc składowej koloru []')
pyplot.ylabel('Liczba pikseli obrazu []')

# Zadanie 2

In [ ]:
def find_closest_palette_color(value, k=2):
    # return numpy.round((k - 1) * value / 255) * 255 / (k - 1)
    return (numpy.round(numpy.multiply(value, (k-1), dtype=numpy.int16) / 255) * 255 / (k-1)).astype(numpy.uint8)


color_reduced = numpy.copy(image_color)
color_reduced = find_closest_palette_color(color_reduced, 4)

#
# Algorytm
#
# for y in range(output.shape[0]):
#     for x in range(output.shape[1]):
#         pass  # TODO: implement

image_color2 = numpy.moveaxis(image_color, -1, 0)
print(image_color2.shape)
dithered_color = numpy.copy(image_color2)
dithered_color[0] = dither_image(image_color2[0], 3)
dithered_color[1] = dither_image(image_color2[1], 3)
dithered_color[2] = dither_image(image_color2[2], 3)

dithered_color = numpy.moveaxis(dithered_color, 0, -1)

#
# Wyświetlenie
#
pyplot.rcParams["figure.figsize"] = (18, 8)
_, plots = pyplot.subplots(1, 2)
plots[0].imshow(color_reduced)
plots[1].imshow(dithered_color)

In [ ]:
#
# Histogram
#
color = ('r', 'g', 'b')

for i, col in enumerate(color):
    histr = cv2.calcHist([dithered_color], [i], None, [256], [0, 256])
    pyplot.plot(histr, color=col)
    pyplot.xlim([-1, 256])
    pyplot.xlabel('Wartośc składowej koloru []')
    pyplot.ylabel('Liczba pikseli obrazu []')

# Zadanie 3

In [ ]:
import math
from itertools import product

#
# Przygotowanie płótna
#
width = 80
height = 60
image = numpy.zeros((height, width, 3), dtype=numpy.uint8)


#
# Funkcja rysująca punkt
#
# NOTE(sdatko): punkt 0,0 to lewy dolny róg obrazu
#
def draw_point(image, x, y, color=(255, 255, 255)):
    image[image.shape[0] - 1 - y, x, :] = color


#
# Funkcja rysująca linię
#
def draw_line(image, x1, y1, x2, y2):
    delta_x = abs(x1 - x2)
    delta_y = abs(y1 - y2)
    xi = numpy.sign(x2 - x1)
    yi = numpy.sign(y2 - y1)
    d = 2 * min(delta_y - delta_x, delta_x - delta_y)
    x = x1
    y = y1
    draw_point(image, x1, y1)
    while x != x2 and y != y2:
        if delta_x > delta_y:
            x += xi
            d += 2 * delta_y
            if d >= 0:
                y += yi
                d -= 2 * delta_x
        else:
            x += xi
            d += 2 * delta_y
            if d >= 0:
                x += xi
                d -= 2 * delta_y

        draw_point(image, x, y)
    draw_point(image, x2, y2)

def area(a, b, c):
    return (c[1] - a[1]) * (b[0] - a[0]) - (c[0] - a[0]) * (b[1] - a[1])

#
# Funkcja rysująca trójkąt
#
def draw_triangle(image, a, b, c):
    verts = numpy.array([a, b, c])
    x_min = numpy.min(verts[:,0])
    y_min = numpy.min(verts[:,1])
    x_max = numpy.max(verts[:,0])
    y_max = numpy.max(verts[:,1])
    for pixel in product(range(x_min, x_max), range(y_min, y_max)):
        areas = numpy.array([
            area(a, b, pixel),
            area(b, c, pixel),
            area(c, a, pixel)
        ])
        signs = areas >= 0
        if numpy.all(signs == signs[0]):
            draw_point(image, pixel[1], pixel[0])


#
# Rysowanie
#
draw_line(image, 10, 10, 60, 40)
draw_triangle(image, (5, 40), (10, 70), (30, 75))

#
# Wyświetlenie
#
pyplot.imshow(image)

# Zadanie 4

In [ ]:
#
# Przygotowanie płótna
#
width = 80
height = 60
image = numpy.zeros((height, width, 3), dtype=numpy.uint8)


#
# Funkcja rysująca punkt
#
# NOTE(sdatko): punkt 0,0 to lewy dolny róg obrazu
#
def draw_point(image, x, y, color=(255, 255, 255)):
    image[image.shape[0] - 1 - y, x, :] = color


def color_blend(color1, color2, t):
    color1 = numpy.array(color1)
    color2 = numpy.array(color2)
    return tuple(color1 + (color2 - color1) * t)

def color_blend_trig(color1, color2, color3, t):
    color1 = numpy.array(color1)
    color2 = numpy.array(color2)
    return tuple(color1 + (color2 - color1) * t)

#
# Funkcja rysująca linię
#
def draw_line(image, x1, y1, x2, y2):
    color1 = (255, 0, 0)
    color2 = (0, 0, 255)

    delta_x = abs(x1 - x2)
    delta_y = abs(y1 - y2)
    xi = numpy.sign(x2 - x1)
    yi = numpy.sign(y2 - y1)
    d = 2 * min(delta_y - delta_x, delta_x - delta_y)
    x = x1
    y = y1
    draw_point(image, x1, y1, color1)
    while x != x2 or y != y2:
        if delta_x > delta_y:
            x += xi
            d += 2 * delta_y
            if d >= 0:
                y += yi
                d -= 2 * delta_x
        else:
            y += yi
            d += 2 * delta_x
            if d >= 0:
                x += xi
                d -= 2 * delta_y

        progress = math.sqrt((x - x1) ** 2 + (y - y1) **2) / math.sqrt(delta_x ** 2 + delta_y ** 2)
        color = color_blend(color1, color2, progress)
        draw_point(image, x, y, color)
    draw_point(image, x2, y2, color2)


#
# Funkcja rysująca trójkąt
#
def draw_triangle(image, a, b, c):
    verts = numpy.array([a, b, c])
    colors = numpy.array([
        [255, 0, 0],
        [0, 255, 0],
        [0, 0, 255]
    ])
    x_min = numpy.min(verts[:,0])
    y_min = numpy.min(verts[:,1])
    x_max = numpy.max(verts[:,0])
    y_max = numpy.max(verts[:,1])
    for pixel in product(range(x_min, x_max), range(y_min, y_max)):
        areas = numpy.array([
            area(a, b, pixel),
            area(b, c, pixel),
            area(c, a, pixel)
        ])
        signs = areas >= 0
        if numpy.all(signs == signs[0]):
            color = numpy.sum(areas / area(a, b, c) * colors, axis=0)
            draw_point(image, pixel[1], pixel[0], color)


#
# Rysowanie
#
draw_line(image, 10, 10, 60, 40)
draw_line(image, 10, 20, 10, 40)
draw_triangle(image, (5, 40), (10, 70), (30, 75))

#
# Wyświetlenie
#
pyplot.imshow(image)

# Zadanie 5

In [ ]:
scale = 2

#
# Przygotowanie płótna
#
width = 80 * scale
height = 60 * scale
image = numpy.zeros((height, width, 3), dtype=numpy.uint8)

# IMO lepiej to zrobić inline niż:
# draw_line(image, 10 * scale, 10 * scale, 60 * scale, 40 * scale)
draw_line(image, 20, 20, 120, 80)
draw_line(image, 20, 40, 20, 80)
draw_triangle(image, (10, 80), (20, 140), (60, 150))

#
# AA
#
parts = numpy.array([image[0::2, 0::2], image[0::2, 1::2], image[1::2, 0::2], image[1::2, 1::2]])
image_aa = numpy.average(parts, axis=0).astype(numpy.uint8)

#
# Wyświetlenie
#
pyplot.imshow(image_aa)